Gerar dataset sintético com regras de fraude

In [87]:
import pandas as pd
import random
import string

# Gera 200 chaves únicas
num_chaves = 200
chaves_pix = set()
while len(chaves_pix) < num_chaves:
    chave = ''.join(random.choices(string.digits, k=11))
    chaves_pix.add(chave)
chaves_pix = list(chaves_pix)

# Associa cada chave a um histórico fixo (fraude ou não)
historico_chaves = {
    chave: random.choice([True, False])
    for chave in chaves_pix
}

ufs = ['SP', 'RJ', 'MG', 'PR', 'RS', 'BA']
tipos_conta = ['corrente', 'poupanca', 'salario']

def gerar_registro():
    chave = random.choice(chaves_pix)
    hist = historico_chaves[chave]
    valor = round(random.uniform(10, 3000), 2)
    hora = random.randint(0, 23)
    local = random.choice(ufs)
    tipo = random.choice(tipos_conta)

    # Regras de fraude
    if hist:
        fraude = 1
    elif 0 <= hora <= 6 and valor > 300:
        fraude = 1
    elif 7 <= hora <= 23 and valor > 2000:
        fraude = 1
    else:
        fraude = 0

    return {
        'valor': valor,
        'hora': hora,
        'localizacao': local,
        'tipo_conta': tipo,
        'chave_pix': chave,
        'chave_historico_fraude': hist,
        'fraude': fraude
    }

N = 10000
df = pd.DataFrame([gerar_registro() for _ in range(N)])
df.to_csv("dados_pix_sinteticos.csv", index=False)
print("✅ CSV gerado com", len(df), "linhas.")

✅ CSV gerado com 10000 linhas.


In [88]:
import pandas as pd

# Carrega o CSV
df = pd.read_csv("dados_pix_sinteticos.csv", dtype={"chave_pix": str})

# Agrupa por chave e conta quantos valores únicos tem para "chave_historico_fraude"
duplicadas = df.groupby("chave_pix")["chave_historico_fraude"].nunique()

# Filtra as que têm mais de um valor (ou seja, True e False ao mesmo tempo)
problemas = duplicadas[duplicadas > 1]

# Exibe as chaves problemáticas (se houver)
if not problemas.empty:
    print("❌ Chaves com marcação inconsistente encontradas:")
    print(problemas)
else:
    print("✅ Nenhuma chave com marcação duplicada encontrada. Tudo certo!")


✅ Nenhuma chave com marcação duplicada encontrada. Tudo certo!


Extrair CSV

In [89]:
from google.colab import files
files.download('dados_pix_sinteticos.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Dicionário de colunas


In [ ]:
column_descriptions = {
    'valor': 'Valor da transação em reais',
    'hora': 'Hora da transação (0–23)',
    'localizacao': 'UF de origem da transação',
    'tipo_conta': 'Tipo de conta (corrente, poupança, salário)',
    'chave_pix': 'Identificador da chave PIX (CPF, e-mail, etc.)',
    'chave_historico_fraude': 'Se a chave já foi identificada em fraude (True/False)',
    'cliente_tem_historico_madrugada': 'Se o cliente já fez PIX na madrugada antes (True/False)',
    'flag_madrugada_incomum': 'Se a transação ocorreu entre 0h e 6h (True/False)',
    'flag_fraude_comportamental': 'Flag combinada de comportamento suspeito (True/False)',
    'fraude': 'Rótulo alvo: 1=fraude, 0=legítima'
}

Pipeline de treinamento (passo a passo)
1. Importar bibliotecas




In [90]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pickle

2. Carregar os dados

In [91]:
df = pd.read_csv("dados_pix_sinteticos.csv")

3. Pré-processamento

In [92]:
# 2) Label encoding manual (mesma codificação do Streamlit)
# 2.5) Aplicar mapeamento manual
map_loc = {'SP':0,'RJ':1,'MG':2,'PR':3,'RS':4,'BA':5}
map_tipo = {'corrente':0,'poupanca':1,'salario':2}

df['localizacao'] = df['localizacao'].map(map_loc)
df['tipo_conta'] = df['tipo_conta'].map(map_tipo)
# Garante que a coluna esteja em formato numérico (0 ou 1)
df["chave_historico_fraude"] = df["chave_historico_fraude"].astype(int)
df["chave_pix"] = df["chave_pix"].astype(str)



4. Dividir variáveis preditoras e alvo

In [93]:
# 3) Features e target
X = df[["valor", "hora", "localizacao", "tipo_conta", "chave_historico_fraude"]]
y = df["fraude"]

5. Separar em treino e teste

In [94]:
# 4) Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

6. Treinar o Random Forest

In [95]:
# 5) Treino
modelo = RandomForestClassifier(n_estimators=100, random_state=42)
modelo.fit(X_train, y_train)
print("✅ Modelo treinado com sucesso!")


✅ Modelo treinado com sucesso!


7. Avaliar desempenho

In [96]:
# 6) Avaliação
y_pred = modelo.predict(X_test)
print("Matriz de Confusão:\n", confusion_matrix(y_test, y_pred))
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))

Matriz de Confusão:
 [[ 474    0]
 [   0 1526]]

Relatório de Classificação:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       474
           1       1.00      1.00      1.00      1526

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000



8. Importância das variáveis

In [97]:
# 7) Importâncias
importancias = pd.Series(modelo.feature_importances_, index=X.columns)
print("\nImportância das features:\n", importancias.sort_values(ascending=False))



Importância das features:
 chave_historico_fraude    0.450151
valor                     0.337133
hora                      0.210465
localizacao               0.001315
tipo_conta                0.000936
dtype: float64


In [98]:
# 8) Salvar o modelo
with open('modelo_random_forest_pix.pkl','wb') as f:
    pickle.dump(modelo, f)
print("✅ modelo_random_forest_pix.pkl salvo com sucesso!")

✅ modelo_random_forest_pix.pkl salvo com sucesso!


In [99]:
from google.colab import files
files.download('modelo_random_forest_pix.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [100]:
import pandas as pd
df = pd.read_csv('dados_pix_sinteticos.csv')
pd.DataFrame(df[df['chave_historico_fraude'] == True]['chave_pix'].unique(),
             columns=['chave_pix'])


,chave_pix
0,65359395034
1,28799064762
2,14307827127
3,19130780869
4,45792592153
...,...
97,64663588335
98,93029957024
99,10177727566
100,87133689075


In [101]:
import pandas as pd

# 1) Leia o CSV
df = pd.read_csv("dados_pix_sinteticos.csv", dtype={"chave_pix": str})

# 2) Liste as chaves marcadas como suspeitas
chaves_suspeitas = df.loc[df['chave_historico_fraude'] == True, 'chave_pix'].unique().tolist()

# 3) Liste as chaves **não** marcadas (legítimas)
chaves_legitimas  = df.loc[df['chave_historico_fraude'] == False, 'chave_pix'].unique().tolist()

print("🔴 Suspeitas:", chaves_suspeitas)
print("🟢 Legítimas:", chaves_legitimas)


🔴 Suspeitas: ['65359395034', '28799064762', '14307827127', '19130780869', '45792592153', '17692725060', '10285221152', '58629150349', '90742287565', '68519683977', '29287476034', '37159394005', '09235067444', '61556028729', '45601621040', '13461210683', '12524583660', '73011702558', '67342987252', '31023511180', '19892177473', '46005520766', '12971737855', '63520066428', '86333340824', '68660923607', '45995363582', '03192263922', '09409571006', '57083290112', '65836105655', '78500669606', '62216586388', '95399207452', '50833850004', '92076109629', '19169000772', '67946617739', '92564139949', '25085304658', '84995116803', '83181669880', '25396619266', '83153281808', '35254229622', '83723909547', '76503735817', '76983676208', '83086204376', '50238649095', '04053280886', '63938194875', '86940505221', '74883987438', '15005709637', '79064286259', '12428018704', '80042974154', '57416196987', '17528240273', '47880396776', '86331739374', '57173182295', '98809589191', '52640027048', '8712932800

In [104]:
df = pd.read_csv("dados_pix_sinteticos.csv", dtype={"chave_pix": str})
df[df["chave_pix"] == "52593521235"]


,valor,hora,localizacao,tipo_conta,chave_pix,chave_historico_fraude,fraude
9,1892.18,15,MG,poupanca,52593521235,False,0
160,13.71,19,PR,poupanca,52593521235,False,0
281,921.40,3,RJ,salario,52593521235,False,1
899,1838.48,0,RS,poupanca,52593521235,False,1
987,161.81,23,RJ,corrente,52593521235,False,0
1061,2651.41,14,PR,corrente,52593521235,False,1
1298,1626.52,22,MG,salario,52593521235,False,0
1342,997.16,8,RJ,poupanca,52593521235,False,0
1348,1455.88,0,BA,poupanca,52593521235,False,1
1407,200.47,7,BA,poupanca,52593521235,False,0
